In [ ]:
# Install the gymnasium package and its dependencies for this notebook.
# !pip install gymnasium
# !pip install gymnasium[toy-text]
# !pip install pygame

import gymnasium as gym

import matplotlib.pyplot as plt
import numpy as np

# Instantiate the Frozen Lake environment. Set is_slippery=False for
# deterministic environment dynamics.
env =gym.make("FrozenLake-v1", render_mode="rgb_array")

# Print the action and observation spaces.
print(f"{env.action_space = }")
print(f"{env.observation_space = } \n")

# Show the numerical number of actions and spaces.
print(f"{env.action_space.n = }")
print(f"{env.observation_space.n = } \n")

# Always reset the environment before taking actions. This sets the
# character in state 0 and returns a dictionary of information concerning
# this initial state.
print(f"{env.reset() = } \n ")

# We can view the game state by calling plt.imshow on the rgb rendering of
# the current game state.
current_grid = env.render()
plt.imshow(current_grid)
plt.show()


---

## Action Space

The action shape is `(1,)` in the range `{0, 3}` indicating which direction to move the player.

    0: Move left

    1: Move down

    2: Move right

    3: Move up



---

In [ ]:
state, reward, done, _, P = env.step(1) # Try and move down.
print(f"{state = }")
print(f"{reward = }")
print(f"{done = }")
print(f"{_ = }")
print(f"{P = }")
new_grid = env.render()
plt.imshow(new_grid)
plt.show()

In [ ]:
state, reward, done, _, P = env.step(1)
print(f"{state = }")
print(f"{reward = }")
print(f"{done = }")
print(f"{_ = }")
print(f"{P = }")
new_grid = env.render()
plt.imshow(new_grid)
plt.show()

In [ ]:
state, reward, done, _, P = env.step(1)
print(f"{state = }")
print(f"{reward = }")
print(f"{done = }")
print(f"{_ = }")
print(f"{P = }")
new_grid = env.render()
plt.imshow(new_grid)
plt.show()

In [ ]:
state, reward, done, _, P = env.step(2)
print(f"{state = }")
print(f"{reward = }")
print(f"{done = }")
print(f"{_ = }")
print(f"{P = }")
new_grid = env.render()
plt.imshow(new_grid)
plt.show()

In [ ]:
state, P = env.reset()
done = False
grids = []
while not done:
    action = env.action_space.sample()
    state, reward, done, _, P = env.step(action)
    new_grid = env.render()
    grids.append(new_grid)


len(grids)

In [ ]:
plt.imshow(grids[0])
plt.show()

In [ ]:
plt.imshow(grids[1])
plt.show()

In [ ]:
plt.imshow(grids[2])
plt.show()

In [ ]:
# Show the transition probabilities for each state-action pair.
print(f"{env.unwrapped.P = }")

In [ ]:
# I need to fix this one.
def run_episodes(environment, n_episodes, policy, display=True):
    wins = 0
    total_reward = 0
    for episode in range(n_episodes):
        terminated = False
        state = environment.reset()
        while not terminated:
            # Select an action to perform in a current state
            if isinstance(policy, str) and policy == 'random':
                action = environment.action_space.sample()
            else:
                action = np.argmax(policy[state])

            # Perform an action and observe how environment acted in response
            next_state, reward, terminated, info = environment.step(action)

            # Plot the first episode
            if episode==1 and display:
                print("Action:")
                plt.imshow(environment.render()) # display current agent state
            # Summarize total reward
            total_reward += reward
            # Update current state
            state = next_state
            # Calculate number of wins over episodes
            if terminated and reward == 1.0:
                wins += 1
    average_reward = total_reward / n_episodes
    return wins, total_reward, average_reward

In [ ]:
def policy_evaluation(policy, environment, discount_factor=0.99, theta=1e-9, max_iterations=1e9):
    # Number of evaluation iterations
    evaluation_iterations = 1
    # Initialize a value function for each state as zero
    V = np.zeros(environment.observation_space.n)
    # Repeat until change in value is below the threshold
    for i in range(int(max_iterations)):
        # Initialize a change of value function as zero
        delta = 0
        # Iterate though each state
        for state in range(environment.observation_space.n):
            v = V[state]
            v_a = []
            for action in range(environment.action_space.n):
                v_x = 0
                for (p, new_state, reward, terminating) in env.unwrapped.P[state][action]:
                    v_x += policy[state][action] * p * (reward + V[new_state])
                v_a.append(v_x)

                # Update value function
            V[state] = max(v_a)

            # Calculate the absolute change of value function
            delta = max(delta, np.abs(V[state] - v))
            # Update value function
            #V[state] = v
        evaluation_iterations += 1

        # Terminate if value change is insignificant
        if delta < theta:
            # TODO - check how many iterations
            print(f'Policy-evaluation converged at iteration #{evaluation_iterations}')
            return V

In [ ]:
pi = {state : {0 : .25, 1 : .25, 2 : .25, 3 : .25} for state in range(env.observation_space.n)}

V = policy_evaluation(pi, env)

In [ ]:
print(f"{V = }")

In [ ]:
def one_step_lookahead(environment, state, V, discount_factor):
    action_values = np.zeros(environment.action_space.n)
    for action in range(environment.action_space.n):
        for probability, next_state, reward, terminated in env.unwrapped.P[state][action]:
            action_values[action] += probability * (reward + discount_factor * V[next_state])
    return action_values

In [ ]:
# Check that the function works!
one_step_lookahead(env, 1, V, 1.0)

In [ ]:
def policy_iteration(environment, discount_factor=1.0, max_iterations=1e9):
    # Start with a random policy
    # num states x num actions / num actions
    policy = np.ones([environment.observation_space.n, environment.action_space.n]) / environment.action_space.n
    # Initialize counter of evaluated policies
    evaluated_policies = 1
    # Repeat until convergence or critical number of iterations reached
    for i in range(int(max_iterations)):
        stable_policy = True
        # Evaluate current policy
        V = policy_evaluation(policy, environment, discount_factor=discount_factor)
        # Go through each state and try to improve actions that were taken (policy Improvement)
        for state in range(environment.observation_space.n):
            # Choose the best action in a current state under current policy
            current_action =  policy[state].tolist().index(max(policy[state]))
            # Look one step ahead and evaluate if current action is optimal
            # We will try every possible action in a current state
            action_value = one_step_lookahead(environment, state, V, discount_factor)
            # Select a better action
            best_action = action_value.tolist().index(max(action_value))
            # If action didn't change
            if current_action != best_action:
                stable_policy = True
                # Greedy policy update
                policy[state] = np.eye(environment.action_space.n)[best_action]
        evaluated_policies += 1
        # If the algorithm converged and policy is not changing anymore, then return final policy and value function
        if stable_policy:
            return policy, V

In [ ]:
policy, V = policy_iteration(env)
print(f"{policy = } \n")
print(f"{V = } \n")

In [ ]:
def value_iteration(environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
    # Initialize state-value function with zeros for each environment state
    V = np.zeros(environment.observation_space.n)
    for i in range(int(max_iterations)):
        # Early stopping condition
        delta = 0
        # Update each state
        for state in range(environment.observation_space.n):
            # Do a one-step lookahead to calculate state-action values
            action_value = one_step_lookahead(environment, state, V, discount_factor)
            # Select best action to perform based on the highest state-action value
            best_action_value = max(action_value)
            # Calculate change in value
            delta = max(delta, np.abs(V[state] - best_action_value))
            # Update the value function for current state
            V[state] = best_action_value
            # Check if we can stop
        if delta < theta:
            print(f'Value-iteration converged at iteration #{i}.')
            break

    # Create a deterministic policy using the optimal value function
    policy = np.zeros([environment.observation_space.n, environment.action_space.n])
    for state in range(environment.observation_space.n):
        # One step lookahead to find the best action for this state
        action_value = one_step_lookahead(environment, state, V, discount_factor)
        # Select best action based on the highest state-action value
        best_action = action_value.tolist().index(max(action_value))
        # Update the policy to perform a better action at a current state
        policy[state, best_action] = 1.0
    return policy, V

In [ ]:
policy, V = value_iteration(env)
print(f"{policy = } \n")
print(f"{V = } \n")